In [1]:
import torch
import pickle
import pandas as pd
from experiment_setup import ExperimentSetup
import plotly.express as px

setup = ExperimentSetup()
setup.netuids = list(setup.netuids)
setup.netuids.remove(11)
setup.netuids.remove(25)

In [2]:
import bittensor as bt

sub = bt.subtensor()

immunity_periods = {}
for netuid in setup.netuids:
    ip = sub.query_subtensor(
        name = 'ImmunityPeriod',
        params = [netuid],
    ).value
    immunity_periods[netuid] = ip 

2024-05-29 14:55:38.153 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-05-29 14:55:38.153 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-05-29 14:55:38.154 |     WARNING      | In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it.
2024-05-29 14:55:38.508 |       INFO       | Connected to finney network and wss://entrypoint-finney.opentensor.ai:443.


In [20]:
div_losts = {}
yuma_results = {}
validator_trusts = {}

for netuid in setup.netuids:
    div_losts[netuid] = {}
    yuma_results[netuid] = {}
    validator_trusts[netuid] = {}
    
    for conceal_period in setup.conceal_periods:
        with open(f"{setup.result_path}/yuma_result_netuid{netuid}_conceal{conceal_period}.pkl", 'rb') as handle:
            _yuma_results = pickle.load(handle)


        dividend = [
            (s["validator_reward_normalized"] / s["stake"]).tolist()
            for idx, s in _yuma_results.items()
        ]

        dividend_df = pd.DataFrame(
            dividend,
            columns=[f"v{i}" for i in range(len(dividend[0]) - 1 )] + ["v_bad"],
            index = _yuma_results.keys()
        )
        
        div_last = dividend_df.iloc[-1]
        div_lost = div_last[-1] / div_last[:-1].median()

        div_losts[netuid][conceal_period] = div_lost
        yuma_results[netuid][conceal_period] = _yuma_results

        v_trust = [
            s['validator_trust'].median()
            for idx, s in _yuma_results.items()
        ]

        validator_trusts[netuid][conceal_period] = sum(v_trust) / len(v_trust)

        # print(
        #     f"netuid: {netuid}",
        #     f"| conceal period (number of tempos): {conceal_period}",
        #     f"| conceal period(hours): {conceal_period * setup.tempo / 300:.2f}",
        #     f"| lost in dividend: {div_lost:.3f}"
        # )

div_losts = pd.DataFrame(div_losts, dtype='float64')
div_losts.index = div_losts.index.map(lambda x : 360 * x)
validator_trusts = pd.DataFrame(validator_trusts, dtype='float64')
validator_trusts.index = validator_trusts.index.map(lambda x : 360 * x)

In [37]:
df = []
for netuid in div_losts:
    min_conceal_period = 0
    for conceal_period, div_lost in div_losts[netuid].items():
        if div_lost > 1:
            if conceal_period == 0:
                min_conceal_period = conceal_period + 1 * 360
            elif conceal_period == 19:
                min_conceal_period = None
            else:
                min_conceal_period = conceal_period + 2 * 360

    if conceal_period != None:
        v_trust = validator_trusts[netuid][0]
    else:
        v_trust = None

    df.append((netuid, min_conceal_period))

min_conceal_period = pd.DataFrame(df, columns = ['netuid', 'commit_reveal_weight_interval'])
min_conceal_period['# 0f tempos'] = min_conceal_period.commit_reveal_weight_interval.map(lambda x : round(x / setup.tempo))
min_conceal_period['hours'] = min_conceal_period.commit_reveal_weight_interval.map(lambda x : x * 12 / 3600)
min_conceal_period = min_conceal_period.set_index('netuid').sort_values('commit_reveal_weight_interval')
min_conceal_period['immunity_period'] = min_conceal_period.index.map(immunity_periods)

min_conceal_period

,commit_reveal_weight_interval,# 0f tempos,hours,immunity_period
netuid,,,,
16,360,1,1.2,14400
2,360,1,1.2,5000
31,360,1,1.2,7200
12,360,1,1.2,2440
10,360,1,1.2,5000
7,1080,3,3.6,10800
32,1080,3,3.6,7200
17,1800,5,6.0,5000
4,2520,7,8.4,5000


In [38]:
with pd.option_context("max_colwidth", 1000):
    print(min_conceal_period.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  commit\_reveal\_weight\_interval &  \# 0f tempos &  hours &  immunity\_period \\
netuid &                                &              &        &                  \\
\midrule
16     &                            360 &            1 &    1.2 &            14400 \\
2      &                            360 &            1 &    1.2 &             5000 \\
31     &                            360 &            1 &    1.2 &             7200 \\
12     &                            360 &            1 &    1.2 &             2440 \\
10     &                            360 &            1 &    1.2 &             5000 \\
7      &                           1080 &            3 &    3.6 &            10800 \\
32     &                           1080 &            3 &    3.6 &             7200 \\
17     &                           1800 &            5 &    6.0 &             5000 \\
4      &                           2520 &            7 &    8.4 &             5000 \\
23     &    

<ipython-input-38-bebfc596c192>:2: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



In [6]:
div_losts.loc[:, [1,2,3]]# .iloc[-1] > 1

,1,2,3
0,1.113691,1.015457,1.090564
360,1.015924,0.995680,1.061116
1080,1.021087,0.987647,1.029612
1800,1.015839,0.985175,1.023372
2520,1.016650,0.985913,1.017134
3240,1.010559,0.985939,1.040942
3960,0.995269,0.985695,1.024334
4680,1.017831,0.984797,0.970292
5400,1.014202,0.984391,0.989544
6120,1.017077,0.984665,1.062407


In [7]:
set_0 = [1, 3, 6, 24, 13]
set_a = (div_losts.iloc[-1] > 1)
set_b = (div_losts.iloc[-1] < 1)

set_a[set_0] = False
set_b[set_0] = False

In [13]:
fig0 = px.line(
    div_losts.loc[:, set_0],
    labels={
        "value": "Relative dividend rate (G)".title(),
        "index": "Commit reveal rate interval",
        "variable": 'Subnet'
    },
    title="Relative Dividend Rate Of Weight Copier <br><sup> For subnets has a varying G that does not monotonically decrease with conceal period</sup>",
    width = 1000,
    height = 500,
)
fig0.add_hline(y=1, line_width=3, line_dash="dash", line_color="red", annotation_text = "")
fig0.update_layout(template='plotly_white')

fig0.add_shape(
    type="rect",
    x0=0,
    x1=19 * 360,
    y0=1.15,
    y1=1,
    fillcolor="red",
    opacity=0.1,
)

fig0.show()

fig1 = px.line(
    div_losts.loc[:, set_a],
    labels={
        "value": "Relative dividend rate (G)".title(),
        "index": "Commit reveal rate interval",
        "variable": 'Subnet'
    },
    title="Relative dividend rate of weight copier <br><sup> For subnets that results in G > 1</sup>".title(),
    width = 1000,
    height = 500,
    range_y = [0.99, 1.06]
)
fig1.add_hline(y=1, line_width=3, line_dash="dash", line_color="red", annotation_text = "")
fig1.update_layout(template='plotly_white')

fig1.add_shape(
    type="rect",
    x0=0,
    x1=19 * 360,
    y0=1,
    y1=1.06,
    fillcolor="red",
    opacity=0.1,
)

fig1.show()

fig2 = px.line(
    div_losts.loc[:, set_b],
    labels={
        "value": "Relative dividend rate (G)".title(),
        "index": "Commit reveal rate interval",
        "variable": 'Subnet'
    },
    title="Relative dividend rate of weight copier <br><sup>For subnets that results in G <= 1</sup>".title(),
    width = 1000,
    height = 500,
    range_y = [0.88, 1.3]
)
fig2.add_hline(y=1, line_width=3, line_dash="dash", line_color="red", annotation_text = "")
fig2.update_layout(template='plotly_white')

fig2.add_shape(
    type="rect",
    x0=0,
    x1=19 * 360,
    y0=1,
    y1=1.3,
    fillcolor="red",
    opacity=0.1,
)

fig2.show()


In [14]:
similarities = {}

for netuid in setup.netuids:
    similarities[netuid] = {}
    
    for conceal_period in setup.conceal_periods:

        if conceal_period == 0:
            similarities[netuid][conceal_period] = 1
            continue

        similarity = torch.load(f"{setup.result_path}/similarity_netuid{netuid}_conceal{conceal_period}.pt")
        similarities[netuid][conceal_period] = similarity.mean().item()

similarities = pd.DataFrame(similarities, dtype='float64')

fig = px.line(
    similarities,
    labels={
        "value": "Similarity",
        "index": "Conceal period (every 360 blocks)",
        "variable": "netuid"
    },
    title="Change in weight measured by cosine similarity".title(),
    width = 1000,
    height = 1000,
)
fig.show()

In [15]:
df = {
    'div_lost': [],
    'similarity': [],
    'netuid': [],
    'correlation': []
}
for i in setup.netuids:
    corr = div_losts[i].corr(similarities[i])
    df['div_lost'] += div_losts[i].tolist()
    df['similarity'] += similarities[i].tolist()
    df['netuid'] += [i] * len(div_losts)
    df['correlation'] += [corr] * len(div_losts)

div_lost_VS_sim = pd.DataFrame(df)
div_lost_VS_sim.netuid = div_lost_VS_sim.netuid.astype('object')

fig = px.scatter(
    div_lost_VS_sim,
    x = 'div_lost',
    y = 'similarity',
    color = 'netuid',
    range_y = [0.7, 1.01],
    range_x = [0.85, 1.32],
    width = 1000,
    height = 1000,
)
fig.show()

/home/isabella/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2634: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/isabella/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2493: RuntimeWarning:

divide by zero encountered in true_divide

/home/isabella/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2634: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/isabella/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2493: RuntimeWarning:

divide by zero encountered in true_divide

/home/isabella/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2634: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/isabella/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2493: RuntimeWarning:

divide by zero encountered in true_divide

/home/isabella/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2634: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/isabella/.local/lib/pytho

In [25]:
17 * 360, 13 * 360, 9 * 360, 5 * 360, 0

(6120, 4680, 3240, 1800, 0)

In [26]:
df = {
    'div_lost': [],
    'validator_trust': [],
    'div_lost': [],
    'validator_trust': [],
    'netuid': [],
}
for i in setup.netuids:
    for conceal_period in [p * 360 for p in [17, 13, 9, 5, 0]]:    
        df['netuid'] += [i]
        df['div_lost'].append(div_losts[i][conceal_period])
        df['validator_trust'].append(validator_trusts[i][conceal_period])

div_lost_VS_validator_trust = pd.DataFrame(df)
div_lost_VS_validator_trust = div_lost_VS_validator_trust.join(
    div_lost_VS_validator_trust.groupby('netuid').validator_trust.mean(), 
    rsuffix = '_mean', 
    on = 'netuid'
)

fig = px.line(
    div_lost_VS_validator_trust,
    x = 'div_lost',
    y = 'validator_trust_mean',
    color = 'netuid',
    labels={
        "div_lost": "Relative dividend rate".title(),
        "validator_trust_mean": "Average Validator Trust".title(),
        "netuid": 'Subnet'
    },
    range_x = [0.85, 1.32],
    width = 1000,
    height = 1000,
    markers = True,
    title ="Change in Relative dividend rate VS validator trust<br><sup>commit reveal weight interval (6120, 4680, 3240, 1800, 0)</sup>".title(),
)
fig.update_layout(template='plotly_white')
fig.show()

In [ ]:
from utils import *
import torch.nn as nn

MSE = nn.MSELoss()

dividend_per_stakes = []
consensus_difference = []
stakes = []

for s in np.arange(0.1, 0.49, 0.05):
    for a in np.arange(0, 1.001, 0.05):
        W = torch.tensor([
            [0, 1],
            [a, 1-a],
        ])
        S = torch.tensor([1-s, s])
        yuma_result = Yuma2(W, S)
        dividend_per_stake = yuma_result['validator_reward_normalized'] / S
        consensus_diff = MSE(yuma_result['server_consensus_weight'], W[1])
        dividend_per_stakes.append(dividend_per_stake[1].item())
        consensus_difference.append(consensus_diff.item())
        stakes.append(s)

dividend_VS_consensus = pd.DataFrame({'dividend_per_stakes': dividend_per_stakes, 'MSE_consensus': consensus_difference, 'stake': stakes})

In [ ]:
dividend_VS_consensus.stake = dividend_VS_consensus.stake.map(lambda x : round(x, 2))

In [ ]:
fig = px.line(
    dividend_VS_consensus,
    y = 'dividend_per_stakes', 
    x = 'MSE_consensus',
    color = 'stake',
    labels={
        "dividend_per_stakes": "Dividend_i per stake",
        "MSE_consensus": "MSE(consensus, weight_i)",
        "stake": 'Stake_i'
    },
    title = "Dividend per stake versus difference in consensus and weight".title(),
    width = 800,
    height = 800,
    # tickformat = '.3f'
)
fig.show()

In [ ]:
yuma_results[24][0].keys()

dict_keys([2987500, 2987860, 2988220, 2988580, 2988940, 2989300, 2989660, 2990020, 2990380, 2990740, 2991100, 2991460, 2991820, 2992180, 2992540, 2992900, 2993260, 2993620, 2993980, 2994340])

In [ ]:
for conceal_period in setup.conceal_periods:
    y1 = list(yuma_results[24][0].items())[-1][1]
    y2 = list(yuma_results[24][conceal_period].items())[-1][1]
    sim = torch.cosine_similarity(y1['server_consensus_weight'].view(1, -1), y2['server_consensus_weight'].view(1, -1))
    print(conceal_period, sim)

0 tensor([1.0000])
1 tensor([0.9984])
3 tensor([0.9896])
5 tensor([0.9850])
7 tensor([0.9832])
9 tensor([0.9814])
11 tensor([0.9795])
13 tensor([0.9741])
15 tensor([0.9758])
17 tensor([0.9679])
19 tensor([0.9652])


In [ ]:
y1['server_consensus_weight'].shape, y2['server_consensus_weight'].shape

(torch.Size([256]), torch.Size([256]))

In [ ]:
y = list(yuma_results[24][conceal_period].items())[-1][1]
DS = y['validator_reward_normalized']/y['stake']

DS[-1] / DS.median()

tensor(0.9858)

In [ ]:
list(yuma_results[24][0].items())[-1][1]

{'weight': tensor([[0.0055, 0.0037, 0.0030,  ..., 0.0039, 0.0000, 0.0049],
         [0.0052, 0.0056, 0.0035,  ..., 0.0037, 0.0000, 0.0027],
         [0.0052, 0.0056, 0.0035,  ..., 0.0037, 0.0000, 0.0027],
         ...,
         [0.0050, 0.0057, 0.0036,  ..., 0.0039, 0.0000, 0.0033],
         [0.0049, 0.0056, 0.0035,  ..., 0.0037, 0.0000, 0.0032],
         [0.0052, 0.0057, 0.0034,  ..., 0.0038, 0.0000, 0.0029]]),
 'stake': tensor([0.0311, 0.0490, 0.1118, 0.0962, 0.0206, 0.0050, 0.0527, 0.0678, 0.0048,
         0.0207, 0.0438, 0.0180, 0.0045, 0.0462, 0.0469, 0.0057, 0.0068, 0.2246,
         0.0865, 0.0196, 0.0083, 0.0085, 0.0207]),
 'server_prerank': tensor([5.2637e-03, 5.5043e-03, 3.4367e-03, 5.5053e-03, 5.1228e-03, 5.0601e-03,
         4.9615e-03, 0.0000e+00, 0.0000e+00, 4.3281e-03, 0.0000e+00, 1.0930e-03,
         5.1467e-03, 4.9071e-03, 3.3759e-03, 5.0608e-03, 5.0241e-03, 4.3618e-03,
         5.3055e-03, 5.5659e-03, 5.1459e-03, 4.7292e-03, 3.9691e-03, 5.3034e-03,
         0.0000e+00,